# Import Event Log

In [1]:
import pandas as pd
import numpy as np
import pm4py
from pm4py.objects.conversion.log import converter as log_converter

if __name__ == "__main__":
    # Read the XES file
    dataframe_log = pm4py.read_xes('../../data/logs/DomesticDeclarations.xes')

    # If 'log' is already a DataFrame, add the @@case_index column directly
    case_indices = {case_id: idx for idx, case_id in enumerate(dataframe_log['case:concept:name'].unique())}
    dataframe_log['@@case_index'] = dataframe_log['case:concept:name'].map(case_indices)
    
     # Convert the dataframe to event log
    log = log_converter.apply(dataframe_log)
    
dataframe_log

/opt/anaconda3/lib/python3.9/site-packages/pm4py/util/dt_parsing/parser.py:76: UserWarning: ISO8601 strings are not fully supported with strpfromiso for Python versions below 3.11
  warnings.warn(


parsing log, completed traces ::   0%|          | 0/10500 [00:00<?, ?it/s]

,id,org:resource,concept:name,time:timestamp,org:role,case:id,case:concept:name,case:BudgetNumber,case:DeclarationNumber,case:Amount,@@case_index
0,st_step 86794_0,STAFF MEMBER,Declaration SUBMITTED by EMPLOYEE,2017-01-09 08:49:50+00:00,EMPLOYEE,declaration 86791,declaration 86791,budget 86566,declaration number 86792,26.851205,0
1,st_step 86793_0,STAFF MEMBER,Declaration FINAL_APPROVED by SUPERVISOR,2017-01-09 10:27:48+00:00,SUPERVISOR,declaration 86791,declaration 86791,budget 86566,declaration number 86792,26.851205,0
2,dd_declaration 86791_19,SYSTEM,Request Payment,2017-01-10 08:34:44+00:00,UNDEFINED,declaration 86791,declaration 86791,budget 86566,declaration number 86792,26.851205,0
3,dd_declaration 86791_20,SYSTEM,Payment Handled,2017-01-12 16:31:22+00:00,UNDEFINED,declaration 86791,declaration 86791,budget 86566,declaration number 86792,26.851205,0
4,st_step 86798_0,STAFF MEMBER,Declaration SUBMITTED by EMPLOYEE,2017-01-09 09:26:14+00:00,EMPLOYEE,declaration 86795,declaration 86795,budget 86566,declaration number 86796,182.464172,1
...,...,...,...,...,...,...,...,...,...,...,...
56432,st_step 138363_0,STAFF MEMBER,Declaration SUBMITTED by EMPLOYEE,2018-12-29 16:50:14+00:00,EMPLOYEE,declaration 138359,declaration 138359,budget 86566,declaration number 138360,190.404576,10499
56433,st_step 138361_0,STAFF MEMBER,Declaration APPROVED by ADMINISTRATION,2018-12-29 16:56:13+00:00,ADMINISTRATION,declaration 138359,declaration 138359,budget 86566,declaration number 138360,190.404576,10499
56434,st_step 138362_0,STAFF MEMBER,Declaration FINAL_APPROVED by SUPERVISOR,2019-01-03 07:55:52+00:00,SUPERVISOR,declaration 138359,declaration 138359,budget 86566,declaration number 138360,190.404576,10499
56435,dd_declaration 138359_19,SYSTEM,Request Payment,2019-01-08 07:20:28+00:00,UNDEFINED,declaration 138359,declaration 138359,budget 86566,declaration number 138360,190.404576,10499


# Drop unnessary columns

In [2]:
dataframe_log = dataframe_log.drop(columns=['case:id'])
dataframe_log = dataframe_log.drop(columns=['case:concept:name'])
dataframe_log = dataframe_log.drop(columns=['case:BudgetNumber'])
dataframe_log = dataframe_log.drop(columns=['case:DeclarationNumber'])
dataframe_log = dataframe_log.drop(columns=['case:Amount'])
dataframe_log = dataframe_log.drop(columns=['id'])

In [3]:
dataframe_log

,org:resource,concept:name,time:timestamp,org:role,@@case_index
0,STAFF MEMBER,Declaration SUBMITTED by EMPLOYEE,2017-01-09 08:49:50+00:00,EMPLOYEE,0
1,STAFF MEMBER,Declaration FINAL_APPROVED by SUPERVISOR,2017-01-09 10:27:48+00:00,SUPERVISOR,0
2,SYSTEM,Request Payment,2017-01-10 08:34:44+00:00,UNDEFINED,0
3,SYSTEM,Payment Handled,2017-01-12 16:31:22+00:00,UNDEFINED,0
4,STAFF MEMBER,Declaration SUBMITTED by EMPLOYEE,2017-01-09 09:26:14+00:00,EMPLOYEE,1
...,...,...,...,...,...
56432,STAFF MEMBER,Declaration SUBMITTED by EMPLOYEE,2018-12-29 16:50:14+00:00,EMPLOYEE,10499
56433,STAFF MEMBER,Declaration APPROVED by ADMINISTRATION,2018-12-29 16:56:13+00:00,ADMINISTRATION,10499
56434,STAFF MEMBER,Declaration FINAL_APPROVED by SUPERVISOR,2019-01-03 07:55:52+00:00,SUPERVISOR,10499
56435,SYSTEM,Request Payment,2019-01-08 07:20:28+00:00,UNDEFINED,10499


# Preprocess

In [4]:
from sklearn.preprocessing import StandardScaler

# Convert to datetime format
dataframe_log['time:timestamp'] = pd.to_datetime(dataframe_log['time:timestamp'])

# Calculate elapsed time since the start of each case
dataframe_log['start_time'] = dataframe_log.groupby('@@case_index')['time:timestamp'].transform('min')
dataframe_log['elapsed_time'] = (dataframe_log['time:timestamp'] - dataframe_log['start_time']).dt.total_seconds()

# Normalize the elapsed time in minutes
scaler = StandardScaler()
dataframe_log['standardized_elapsed_time'] = scaler.fit_transform(dataframe_log[['elapsed_time']])

dataframe_log = dataframe_log.drop(columns=['start_time'])
dataframe_log = dataframe_log.drop(columns=['elapsed_time'])
dataframe_log = dataframe_log.drop(columns=['time:timestamp'])

In [5]:
codes, uniques = pd.factorize(dataframe_log['concept:name'])
dataframe_log['concept:name'] = codes + 1

In [6]:
codes, uniques = pd.factorize(dataframe_log['org:resource'])
dataframe_log['org:resource'] = codes + 1

In [7]:
codes, uniques = pd.factorize(dataframe_log['org:role'])
dataframe_log['org:role'] = codes + 1

In [8]:
dataframe_log

,org:resource,concept:name,org:role,@@case_index,standardized_elapsed_time
0,1,1,1,0,-0.372192
1,1,2,2,0,-0.367752
2,2,3,3,0,-0.307612
3,2,4,3,0,-0.155482
4,1,1,1,1,-0.372192
...,...,...,...,...,...
56432,1,1,1,10499,-0.372192
56433,1,11,6,10499,-0.371921
56434,1,2,2,10499,-0.070090
56435,2,3,3,10499,0.254628


# Generate Prefixes

In [9]:
df_resource = dataframe_log[['org:resource', '@@case_index']]
df_activity = dataframe_log[['concept:name', '@@case_index']]
df_role = dataframe_log[['org:role', '@@case_index']]
df_timestamp = dataframe_log[['standardized_elapsed_time', '@@case_index']]

In [10]:
def generate_sliding_windows(df, case_id_column='@@case_index', window_size=5):
    windows = []
    targets = []
    case_indices = []

    # Iterate over each unique case
    for case_id in df[case_id_column].unique():
        # Extract the case
        case_data = df[df[case_id_column] == case_id]
        
        # Convert case_data to a NumPy array and drop the case_id_column
        case_data_array = case_data.drop(columns=[case_id_column]).to_numpy()

        # Adjusting the condition to correctly reflect window_size without needing an additional +1
        # Now it correctly considers window_size as including the target event
        if len(case_data_array) >= window_size:
            # Adjust the loop to generate sliding windows of size window_size - 1 for the inputs and use the next event as the target
            for i in range(len(case_data_array) - window_size + 1):
                # window now has window_size - 1 events
                window = case_data_array[i:i + window_size - 1]
                # The target is the event immediately following the window
                target = case_data_array[i + window_size - 1]
                windows.append(window)
                targets.append(target)
                case_indices.append(case_id)  # Store the case_id corresponding to the window

    # Convert lists to numpy arrays for easier handling and to ensure they are two-dimensional
    windows_array = np.array(windows)
    targets_array = np.array(targets)
    case_indices_array = np.array(case_indices)
    
    return windows_array, targets_array, case_indices_array

In [11]:
windows_resource, targets_resource, case_indices = generate_sliding_windows(df_resource)
windows_activity, targets_activity, case_indices = generate_sliding_windows(df_activity)
windows_role, targets_role, case_indices = generate_sliding_windows(df_role)
windows_timestamp, targets_timestamp, case_indices = generate_sliding_windows(df_timestamp)

# GRU

### Architecture

- Separate Inputs for Each Attribute
- Each attribute is passed through an embedding layer
- Each attribute has its corresponding GRU encoder
- Selective Concatenation: After encoding, the outputs of these GRU layers are concatenated. However, this concatenation is selective, meaning it is structured in a way that prepares the data for effective synthesis without leaking information from the future (next event attributes)
- Decoder GRUs: Integrated Decoding: Post-concatenation, the combined attributes are processed through decoder GRU layers. These layers are tasked with integrating the data from different attributes and preparing it for final prediction. This step is where BINet v3 distinguishes itself by effectively using the interdependencies between different attributes to enhance prediction accuracy.
- Output Layer: Softmax Output for Each Attribute: For each attribute of the next event, a softmax layer predicts a probability distribution over all possible values. This allows the model to output the most likely next event and its attributes based on the learned dependencies and the history encoded by the GRUs.
- E: maximum case length
- We train BINet with a GRU size of 2E (two times the maximum case length)
- on mini batches of size 500 for 20 epochs

In [12]:
# Group by the @@case_index column and count the rows in each group
case_lengths = dataframe_log.groupby('@@case_index').size()

# Find the maximum value among the case lengths
E = case_lengths.max()

In [13]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GRU, Embedding, Dense, Dropout, Concatenate, BatchNormalization

def create_binetv3(num_resources, num_activities, num_roles, embedding_dim, gru_units, dropout_rate):
    # Input layers for each attribute
    input_resource = Input(shape=(None,), name='resource_input')
    input_activity = Input(shape=(None,), name='activity_input')
    input_role = Input(shape=(None,), name='role_input')
    input_timestamp = Input(shape=(None, 1), name='timestamp_input')

    # Embedding layers for categorical attributes
    embedding_resource = Embedding(input_dim=num_resources + 1, output_dim=embedding_dim, mask_zero=True, name='resource_embedding')(input_resource)
    embedding_activity = Embedding(input_dim=num_activities + 1, output_dim=embedding_dim, mask_zero=True, name='activity_embedding')(input_activity)
    embedding_role = Embedding(input_dim=num_roles + 1, output_dim=embedding_dim, mask_zero=True, name='role_embedding')(input_role)

    # Encoder GRUs with Batch Normalization for categorical attributes
    encoded_resource = GRU(units=gru_units, return_sequences=True, name='resource_encoder')(embedding_resource)
    bn_resource = BatchNormalization(name='bn_resource')(encoded_resource)
    encoded_activity = GRU(units=gru_units, return_sequences=True, name='activity_encoder')(embedding_activity)
    bn_activity = BatchNormalization(name='bn_activity')(encoded_activity)
    encoded_role = GRU(units=gru_units, return_sequences=True, name='role_encoder')(embedding_role)
    bn_role = BatchNormalization(name='bn_role')(encoded_role)

    # Encoder GRU with Batch Normalization for continuous attribute
    encoded_timestamp = GRU(units=gru_units, return_sequences=True, name='timestamp_encoder')(input_timestamp)
    bn_timestamp = BatchNormalization(name='bn_timestamp')(encoded_timestamp)

    # Concatenation of encoded outputs
    concatenated = Concatenate(name='concatenate_encodings')([bn_resource, bn_activity, bn_role, bn_timestamp])

    # Decoder GRU
    decoder_output = GRU(units=gru_units, return_sequences=False, name='decoder_gru')(concatenated)
    dropout_layer = Dropout(rate=dropout_rate, name='dropout')(decoder_output)

    # Output layers for predicting the next event's attributes
    output_resource = Dense(num_resources + 1, activation='softmax', name='output_resource')(dropout_layer)
    output_activity = Dense(num_activities + 1, activation='softmax', name='output_activity')(dropout_layer)
    output_role = Dense(num_roles + 1, activation='softmax', name='output_role')(dropout_layer)
    output_timestamp = Dense(1, activation='linear', name='output_timestamp')(dropout_layer)

    # Building the model
    model = Model(inputs=[input_resource, input_activity, input_role, input_timestamp], outputs=[output_resource, output_activity, output_role, output_timestamp])
    model.compile(
        optimizer='adam', 
        loss={
            'output_resource': 'categorical_crossentropy', 
            'output_activity': 'categorical_crossentropy',
            'output_role': 'categorical_crossentropy',
            'output_timestamp': 'mse'
        },
        metrics={
            'output_resource': ['accuracy'], 
            'output_activity': ['accuracy'],
            'output_role': ['accuracy'],
            'output_timestamp': ['mse']
        }
    )

    return model

# Parameters
gru_units = int(2 * E) 
num_resources = dataframe_log['org:resource'].max()
num_activities = dataframe_log['concept:name'].max()
num_roles = dataframe_log['org:role'].max()
embedding_dim = 50
dropout_rate = 0.2
model = create_binetv3(num_resources, num_activities, num_roles, embedding_dim, gru_units, dropout_rate)
model.summary()

2024-08-18 17:58:35.575227: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 resource_input (InputLayer  [(None, None)]               0         []                            
 )                                                                                                
                                                                                                  
 activity_input (InputLayer  [(None, None)]               0         []                            
 )                                                                                                
                                                                                                  
 role_input (InputLayer)     [(None, None)]               0         []                            
                                                                                              

### Data Splitting

In [14]:
from sklearn.model_selection import train_test_split

train_resource, test_resource, train_targets_resource, test_targets_resource = train_test_split(
    windows_resource, targets_resource, test_size=0.3, random_state=42)

train_activity, test_activity, train_targets_activity, test_targets_activity = train_test_split(
    windows_activity, targets_activity, test_size=0.3, random_state=42)

train_role, test_role, train_targets_role, test_targets_role = train_test_split(
    windows_role, targets_role, test_size=0.3, random_state=42)

train_timestamp, test_timestamp, train_targets_timestamp, test_targets_timestamp = train_test_split(
    windows_timestamp, targets_timestamp, test_size=0.3, random_state=42)

### Training

In [15]:
from tensorflow.keras.utils import to_categorical

# One-hot encode categorical targets
train_targets_resource_cat = to_categorical(train_targets_resource, num_classes=num_resources + 1)
test_targets_resource_cat = to_categorical(test_targets_resource, num_classes=num_resources + 1)

train_targets_activity_cat = to_categorical(train_targets_activity, num_classes=num_activities + 1)
test_targets_activity_cat = to_categorical(test_targets_activity, num_classes=num_activities + 1)

train_targets_role_cat = to_categorical(train_targets_role, num_classes=num_roles + 1)
test_targets_role_cat = to_categorical(test_targets_role, num_classes=num_roles + 1)

In [16]:
# Fit the model
history = model.fit(
    [train_resource, train_activity, train_role, train_timestamp],
    [train_targets_resource_cat, train_targets_activity_cat, train_targets_role_cat, train_targets_timestamp],
    validation_data=(
        [test_resource, test_activity, test_role, test_timestamp], 
        [test_targets_resource_cat, test_targets_activity_cat, test_targets_role_cat, test_targets_timestamp]
    ),
    epochs=20,
    batch_size=500
)

Epoch 1/20
22/22 [==============================] - 25s 299ms/step - loss: 4.3429 - output_resource_loss: 0.4649 - output_activity_loss: 1.7535 - output_role_loss: 1.0977 - output_timestamp_loss: 1.0268 - output_resource_accuracy: 0.8686 - output_activity_accuracy: 0.6764 - output_role_accuracy: 0.7061 - output_timestamp_mse: 1.0268 - val_loss: 6.2224 - val_output_resource_loss: 0.6912 - val_output_activity_loss: 2.3139 - val_output_role_loss: 1.3095 - val_output_timestamp_loss: 1.9078 - val_output_resource_accuracy: 0.8004 - val_output_activity_accuracy: 0.5680 - val_output_role_accuracy: 0.7999 - val_output_timestamp_mse: 1.9078
Epoch 2/20
22/22 [==============================] - 1s 55ms/step - loss: 1.8597 - output_resource_loss: 0.0927 - output_activity_loss: 0.6156 - output_role_loss: 0.4056 - output_timestamp_loss: 0.7458 - output_resource_accuracy: 0.9885 - output_activity_accuracy: 0.8563 - output_role_accuracy: 0.8778 - output_timestamp_mse: 0.7458 - val_loss: 5.5153 - val_out

In [17]:
# Evaluate the model on the validation set
results = model.evaluate(
    [test_resource, test_activity, test_role, test_timestamp],
    [test_targets_resource_cat, test_targets_activity_cat, test_targets_role_cat, test_targets_timestamp],
    batch_size=64
)
print(f"Validation Loss: {results[0]}, Validation Accuracy: {results[1]}")

71/71 [==============================] - 1s 8ms/step - loss: 0.9241 - output_resource_loss: 0.0270 - output_activity_loss: 0.2498 - output_role_loss: 0.1185 - output_timestamp_loss: 0.5288 - output_resource_accuracy: 0.9987 - output_activity_accuracy: 0.9602 - output_role_accuracy: 0.9673 - output_timestamp_mse: 0.5288
Validation Loss: 0.9241350293159485, Validation Accuracy: 0.027006646618247032


In [40]:
# Save the model to an H5 file
model.save('BINet_V1_Domestic.h5')

/opt/anaconda3/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Anomaly Score Computation

- For each event attribute, BINet's softmax layer outputs a probability distribution over possible values
- The anomaly score for a specific attribute value v is calculated by summing all the probabilities from the softmax output that are greater than the probability assigned to v

In [19]:
# Generate predictions for all inputs
predictions = model.predict([windows_resource, windows_activity, windows_role, windows_timestamp])


# Assuming your model is set to predict resource, activity, and role categories
# Extract predictions for categorical attributes (softmax probabilities)
predictions_resource = predictions[0]  # Activity predictions
predictions_activity = predictions[1]  # Resource predictions
predictions_role = predictions[2]      # Role predictions
# If you had added 'amount' as a target to be predicted, you would extract its predictions like so:
predictions_timestamp = predictions[3]  # Assuming 'amount' is a regression target and the model is adjusted accordingly

472/472 [==============================] - 9s 3ms/step


In [20]:
import numpy as np

def calculate_anomaly_scores(predictions, targets):
    scores = []
    # Loop through each example in the predictions
    for i in range(predictions.shape[0]):
        actual_prob = predictions[i, targets[i]]  # Extract the probability of the true class using target index
        # Calculate anomaly score as sum of probabilities greater than the probability of the actual value
        anomaly_score = np.sum(predictions[i][predictions[i] > actual_prob])
        scores.append(anomaly_score)

    return scores

In [21]:
# Calculate anomaly scores for each attribute type
anomaly_scores_resource = calculate_anomaly_scores(predictions_resource, targets_resource.astype(int))
anomaly_scores_activity = calculate_anomaly_scores(predictions_activity, targets_activity.astype(int))
anomaly_scores_role = calculate_anomaly_scores(predictions_role, targets_role.astype(int))

In [22]:
def compute_anomaly_scores_continuous(predictions, actuals):
    # Calculate absolute differences
    differences = np.abs(predictions - actuals)
    
    # Normalize to [0, 1] range
    max_diff = np.max(differences)
    normalized_scores = differences / max_diff if max_diff != 0 else differences
    
    return normalized_scores

In [23]:
anomaly_scores_timestamp = compute_anomaly_scores_continuous(predictions_timestamp, targets_timestamp)
anomaly_scores_timestamp = anomaly_scores_timestamp.flatten()
anomaly_scores_timestamp = anomaly_scores_timestamp.tolist()

## Insert missing scores for cases with less than 2 Events

In [24]:
import pandas as pd

# Create a DataFrame from the case_indices_array corresponding to case_resource
score = pd.DataFrame({'case': case_indices})
score['score_resource'] = anomaly_scores_resource
score['score_activity'] = anomaly_scores_activity
score['score_role'] = anomaly_scores_role
score['score_timestamp'] = anomaly_scores_timestamp

score

,case,score_resource,score_activity,score_role,score_timestamp
0,1,0.0,0.0,0.0,0.101453
1,2,0.0,0.0,0.0,0.043343
2,6,0.0,0.0,0.0,0.014467
3,6,0.0,0.0,0.0,0.028962
4,6,0.0,0.0,0.0,0.020011
...,...,...,...,...,...
15070,10496,0.0,0.0,0.0,0.012635
15071,10496,0.0,0.0,0.0,0.017335
15072,10497,0.0,0.0,0.0,0.005612
15073,10498,0.0,0.0,0.0,0.005585


In [25]:
import pandas as pd

def contains_all_values(df, column, end):

    # Generate the set of all values in the specified range
    required_values = set(range(1, end))
    
    # Get the unique values in the specified column
    column_values = set(df[column].unique())
    
    # Find missing values
    missing_values = required_values - column_values
    
    # Print missing values if any
    if missing_values:
        print(f"Missing values: {sorted(missing_values)}")
    
    # Check if all required values are in the column values
    return required_values.issubset(column_values)

end = 10499

result = contains_all_values(score, 'case', end)
print(f"Does the 'case' column contain all values between 0 and {end}? {result}")

Missing values: [3, 4, 5, 7, 8, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 38, 39, 40, 41, 42, 43, 44, 46, 52, 53, 54, 55, 56, 57, 58, 60, 61, 62, 63, 66, 67, 70, 71, 72, 73, 74, 75, 77, 79, 80, 81, 83, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 110, 112, 113, 114, 115, 116, 118, 120, 121, 122, 125, 126, 127, 129, 130, 131, 132, 133, 135, 136, 137, 138, 140, 142, 143, 144, 145, 146, 149, 150, 152, 154, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 168, 169, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 182, 185, 186, 187, 188, 189, 190, 191, 192, 194, 196, 197, 198, 199, 200, 202, 203, 204, 205, 206, 207, 209, 210, 211, 212, 213, 214, 215, 216, 218, 221, 225, 226, 227, 228, 230, 231, 232, 233, 234, 235, 236, 237, 238, 240, 241, 243, 245, 246, 248, 249, 250, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 269, 270, 271, 272, 273, 275, 279, 28

In [26]:
# Generate the full range of values
full_range = set(range(10500))

# Get the existing 'case' values
existing_cases = set(score['case'])

# Identify missing values
missing_cases = full_range - existing_cases

# Create a DataFrame for missing values
missing_df = pd.DataFrame({'case': list(missing_cases)})

# Add other columns with value 1
for col in score.columns:
    if col != 'case':
        missing_df[col] = 1

# Concatenate the original DataFrame with the missing values DataFrame
updated_score = pd.concat([score, missing_df])

# Sort the DataFrame by 'case'
updated_score = updated_score.sort_values(by='case').reset_index(drop=True)

updated_score

,case,score_resource,score_activity,score_role,score_timestamp
0,0,1.0,1.0,1.0,1.000000
1,1,0.0,0.0,0.0,0.101453
2,2,0.0,0.0,0.0,0.043343
3,3,1.0,1.0,1.0,1.000000
4,4,1.0,1.0,1.0,1.000000
...,...,...,...,...,...
16879,10496,0.0,0.0,0.0,0.012635
16880,10496,0.0,0.0,0.0,0.017335
16881,10497,0.0,0.0,0.0,0.005612
16882,10498,0.0,0.0,0.0,0.005585


In [27]:
anomaly_scores_resource = updated_score['score_resource']
anomaly_scores_activity = updated_score['score_activity']
anomaly_scores_role = updated_score['score_role']
anomaly_scores_timestamp = updated_score['score_timestamp']

### Threshold (lowest plateau)

In [28]:
import numpy as np

def calculate_anomaly_ratio(scores, threshold):
    """
    Calculate the anomaly ratio for a given threshold.
    """
    return np.mean(scores > threshold)

def find_plateaus(scores, epsilon=1e-4, min_plateau_length=10):
    """
    Identify the lowest plateau in the anomaly ratio function and calculate the mean-centered threshold.
    """
    scores = np.array(scores)  # Convert scores to a NumPy array
    sorted_scores = np.sort(scores)
    
    # Remove duplicate values
    unique_thresholds, unique_indices = np.unique(sorted_scores, return_index=True)
    anomaly_ratios = np.array([calculate_anomaly_ratio(scores, t) for t in unique_thresholds])
    
    # Calculate first and second derivatives
    first_derivatives = np.diff(anomaly_ratios) / np.diff(unique_thresholds)
    second_derivatives = np.diff(first_derivatives) / np.diff(unique_thresholds[:-1])
    
    # Identify plateaus where the first derivative is close to zero
    plateau_indices = np.where(np.abs(first_derivatives) < epsilon)[0]
    
    # Group consecutive indices to identify continuous plateaus
    grouped_plateaus = np.split(plateau_indices, np.where(np.diff(plateau_indices) != 1)[0] + 1)
    
    # Filter plateaus based on minimum length
    long_plateaus = [g for g in grouped_plateaus if len(g) >= min_plateau_length]
    
    if long_plateaus:
        # Take the first long plateau and find the mean threshold in this plateau
        first_plateau = long_plateaus[0]
        plateau_thresholds = unique_thresholds[first_plateau]
        return np.mean(plateau_thresholds)
    else:
        # If no plateau is found, return a default value, e.g., the 90th percentile
        percentile_90 = np.percentile(sorted_scores, 90)
        if percentile_90 == 1.0:
            return 0.4
        else:
            return percentile_90

In [29]:
threshold_resource = find_plateaus(anomaly_scores_resource)
threshold_role = find_plateaus(anomaly_scores_role)
threshold_activity = find_plateaus(anomaly_scores_activity)
threshold_timestamp = find_plateaus(anomaly_scores_timestamp)

### Detection

In [30]:
def detect_anomalies(anomaly_scores, threshold):
    labels = [1 if score > threshold else 0 for score in anomaly_scores]
    return labels

In [31]:
# Detect anomalies based on the calculated anomaly scores and thresholds
labels_resource = detect_anomalies(anomaly_scores_resource, threshold_resource)
labels_activity = detect_anomalies(anomaly_scores_activity, threshold_activity)
labels_role = detect_anomalies(anomaly_scores_role, threshold_role)
labels_timestamp = detect_anomalies(anomaly_scores_timestamp, threshold_timestamp)

# Mapping

In [32]:
case_indices = updated_score['case']

In [33]:
import pandas as pd

# Create a DataFrame from the case_indices_array corresponding to case_resource
mapping = pd.DataFrame({'case': case_indices})
mapping['predicted_resource'] = labels_resource
mapping['predicted_activity'] = labels_activity
mapping['predicted_role'] = labels_role
mapping['predicted_timestamp'] = labels_timestamp

mapping

,case,predicted_resource,predicted_activity,predicted_role,predicted_timestamp
0,0,1,1,1,1
1,1,0,0,0,0
2,2,0,0,0,0
3,3,1,1,1,1
4,4,1,1,1,1
...,...,...,...,...,...
16879,10496,0,0,0,0
16880,10496,0,0,0,0
16881,10497,0,0,0,0
16882,10498,0,0,0,0


In [34]:
# Create a boolean DataFrame where each value is True if the value is 1
contains_one = (mapping[['predicted_resource', 'predicted_activity', 'predicted_role', 'predicted_timestamp']] == 1)

# Group by 'case' and check if there's at least one 'True' in any of the columns
case_prediction = contains_one.groupby(mapping['case']).any().any(axis=1)
case_prediction

case
0         True
1        False
2        False
3         True
4         True
         ...  
10495    False
10496    False
10497    False
10498    False
10499    False
Length: 10500, dtype: bool

# Ground Truth

In [35]:
def generate_alignments_adjusted_tracecost_pkl(log, net, initial_marking, final_marking):
    from pm4py.algo.conformance.alignments.petri_net import algorithm as alignments
    from pm4py.algo.conformance.alignments.petri_net import variants
    from pm4py.objects.petri_net.utils import align_utils
    max_events=0
    for trace in log:
        counter=0
        for event in trace:
            counter+=1
        if counter > max_events:
            max_events=counter
    parameters={}
    parameters[alignments.Variants.VERSION_STATE_EQUATION_A_STAR.value.Parameters.PARAM_SYNC_COST_FUNCTION] = list(map(lambda i: .1*i, range(max_events*2)))
    parameters[alignments.Variants.VERSION_STATE_EQUATION_A_STAR.value.Parameters.PARAM_TRACE_COST_FUNCTION]=list(map(lambda i: align_utils.STD_MODEL_LOG_MOVE_COST-.1*i, range(max_events*2)))
    aligned_traces = alignments.apply_log(log, net, initial_marking, final_marking, variant=variants.state_equation_a_star, parameters=parameters)
    return aligned_traces

In [36]:
import pm4py
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.objects.bpmn.importer import importer as bpmn_importer
from pm4py.algo.conformance.alignments.petri_net import algorithm as alignments_petri

# 2. Import the given BPMN model
bpmn_graph = bpmn_importer.apply("../../data/model/Model_ DomesticDeclarations.bpmn")

# 3. Convert the BPMN to a Petri net
net, initial_marking, final_marking = pm4py.convert_to_petri_net(bpmn_graph)

aligned_traces = generate_alignments_adjusted_tracecost_pkl(log, net, initial_marking, final_marking)

aligning log, completed variants ::   0%|          | 0/99 [00:00<?, ?it/s]

In [37]:
def extract_conformance_status_by_fitness(aligned_traces):
    conformance_status = []
    for alignment in aligned_traces:
        fitness = alignment['fitness']
        # If the fitness is 1.0, the trace is conforming
        if fitness == 1.0:
            conformance_status.append(0)
        else:
            conformance_status.append(1)
    return conformance_status

# Get the conformance status list from the aligned traces
conformance = extract_conformance_status_by_fitness(aligned_traces)

In [46]:
ground_truth = pd.DataFrame({'conformity': conformance})
ground_truth['predicted'] = case_prediction

# Convert False to 0 and True to 1
ground_truth['predicted'] = [int(value) for value in ground_truth['predicted']]
ground_truth

,conformity,predicted
0,1,1
1,1,0
2,1,0
3,1,1
4,1,1
...,...,...
10495,0,0
10496,0,0
10497,0,0
10498,0,0


# Evaluation

In [47]:
# Calculating TP, TN, FP, FN
TP = ((ground_truth['conformity'] == 1) & (ground_truth['predicted'] == 1)).sum()
TN = ((ground_truth['conformity'] == 0) & (ground_truth['predicted'] == 0)).sum()
FP = ((ground_truth['conformity'] == 0) & (ground_truth['predicted'] == 1)).sum()
FN = ((ground_truth['conformity'] == 1) & (ground_truth['predicted'] == 0)).sum()

In [48]:
precision_dev = TP / (TP + FP)
print(f"Precision Dev: {precision_dev:.2f}")

Precision Dev: 0.91


In [49]:
recall_dev = TP / (TP + FN)
print(f"Recall Dev: {recall_dev:.2f}")

Recall Dev: 0.67


In [50]:
precision_no_dev = TN / (TN + FN)
print(f"Precision No Dev: {precision_no_dev:.2f}")

Precision No Dev: 0.87


In [51]:
recall_no_dev = TN / (TN + FP)
print(f"Recall No Dev: {recall_no_dev:.2f}")

Recall No Dev: 0.97


In [52]:
from sklearn.metrics import roc_auc_score

auc_roc = roc_auc_score(ground_truth['conformity'], ground_truth['predicted'])
print(f"AUC-ROC: {auc_roc:.2f}")

AUC-ROC: 0.82
